## 参照


+ Botterのためのデータ可視化入門 — Botterのためのデータ可視化入門 - https://botter4visualization.readthedocs.io/en/latest/intro.html
+ ETF SPDR S&P 500 ETF Trust 過去データ - Investing.com - https://jp.investing.com/etfs/spdr-s-p-500-historical-data
+ S&P500 過去のレート - Investing.com - https://jp.investing.com/indices/us-spx-500-historical-data
+ S&P 500 TR 過去のレート - Investing.com - https://jp.investing.com/indices/s-p-500-tr-historical-data




In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import japanize_matplotlib

## 前準備

In [ ]:
# investingcom から取得した月次データをDataframeへ変換
def to_df(f):
    df = pd.read_csv(f, header=[0])
    df = df.drop(["出来高","変化率 %"], axis=1)

    df["日付け"] = pd.to_datetime(df["日付け"])
    df.set_index("日付け", inplace=True)
    df = df.sort_index()
    try:
        df = df.applymap(lambda x: pd.to_numeric(x))
    except ValueError:
        df = df.applymap(lambda x: x.replace(",", "")).applymap(pd.to_numeric)
    
    return df 

In [ ]:
# simulation 

def sim(df, df_fx, unit, cost):
    # unit = 30000
    # cost = 0.75/100

    df = df.merge(df_fx["終値"], on="日付け", suffixes=["","_fx"], how="left")
    df["基準価額"] = df["終値"] * df["終値_fx"]
    df["基準価額変化率"] = df["基準価額"].pct_change().cumsum() + 1 
    df["購入口数"] = unit / df["基準価額"] 
    df["総保有口数"] = df["購入口数"].cumsum()
    df["資産評価額"] = df["基準価額"] * df["総保有口数"] 
    df["投資元本"] = [unit * (1 - cost)] * len(df)
    df["投資元本"] = df["投資元本"].cumsum()    
    df["損益"] = df["資産評価額"] - df["投資元本"]
    df["損益%"] = df["資産評価額"] / df["投資元本"] 
    df["評価額変化率"] = df["資産評価額"].pct_change()

    return df 


In [ ]:
# Dataframe 用意
df_sp500 = to_df("../data/sp500.csv")
df_sp500tr = to_df("../data/sp500tr.csv")
df_spy = to_df("../data/spy.csv")
df_usdjpy = to_df("../data/usdjpy.csv")
df_n225 = to_df("../data/n225.csv")


# simulation 計算に為替が不要のときに使う
df_jpy = df_usdjpy.applymap(lambda x: 1)



## 運用シュミュレーション
+ SP500ETF（SPY）とSP500配当再投資型を積立投資した場合

In [ ]:
unit = 1
cost = 0.75/100

df_sim_index = sim(df_sp500, df_usdjpy, unit, cost)   
df_sim_tr = sim(df_sp500tr, df_usdjpy, unit, cost)   
df_sim_spy = sim(df_spy, df_usdjpy, unit, cost)   

In [ ]:
df_merged = pd.DataFrame(
    {
    "INDEX": df_sim_index["評価額変化率"],
    "SP500TR": df_sim_tr["評価額変化率"],
    "SPY": df_sim_spy["評価額変化率"],
})

In [ ]:
df_merged.describe()

In [ ]:
pd.DataFrame({
    "SP500TR PL": df_sim_tr["損益"],
    "SPY PL": df_sim_spy["損益"],
    "INDEX Close": df_sim_index["終値"]
    }).plot(secondary_y="INDEX Close", grid=True)

## 一定期間を運用した場合のSimulation

データが2000年1月〜直近まであるので、ランダムにスタート地点をN個選び、そこからT期間運用した場合のSimulation



In [ ]:
def random_sim(df_target,df_fx, n, T):
    results = list()
    for dt in df_target.index[:-T].to_series().sample(frac=1)[:n]:
        _df = df_target.loc[dt:].head(T)
        _df = sim(_df, df_fx, 1, 0.75/100)
        s = pd.Series(_df["損益%"].to_list())
        s.name = dt.strftime("%Y/%m")
        results.append(s)
    return pd.DataFrame(results).T
        
def random_sim_plot(df_target, df_fx,n, T):
    df = random_sim(df_target, df_fx, n, T)
    ax = df.plot(figsize=(15, 4), legend=True, grid=True, title=f"期間：{T}ヶ月 / 回数 {n} 回", ylim=(0, 3),)
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    ax.axhline(y=1, color="black", )

In [ ]:
n = 20
T = 12 * 10 # 12ヶ月 * 10 年
df_target = df_spy
random_sim_plot(df_target, df_usdjpy, n, T)

In [ ]:
n = 20
T = 12 * 10 # 12ヶ月 * 10 年
df_target = df_n225
random_sim_plot(df_target, df_jpy, n, T)

In [ ]:
def random_sim2(df_target, df_fx, n, T):
    results = list()
    for dt in df_target.index[:-T].to_series().sample(frac=1)[:n].sort_index():
        _df = df_target.loc[dt:].head(T)
        _df = sim(_df, df_fx, 1, 0.75/100)
        results.append(_df)
    return results

def random_sim2_plot(results, n, T):
    fig = plt.figure(figsize=(18, 10))
    i = 1
    for _df in results:
        ax = fig.add_subplot(4, 5, i)
        _df[["基準価額変化率","損益%"]].plot(
            ax=ax,
            ylim=(0, 3),
            rot=45,
            grid=True,
            title = _df.index[0].strftime("start: %Y/%m")
            )
        i += 1
    plt.tight_layout()    

In [ ]:
n = 20
T = 12 * 10 # 12ヶ月 * 10 年
df_target = df_spy

results = random_sim2(df_target, df_usdjpy,  n, T)
random_sim2_plot(results, n, T)
